
# Numerical integration and Richardson extrapolation


Trapezoidal rule with the integration step $h$:

\begin{align*}
    \int_a^b f(x) \, \mathrm{d}x 
    & =
    \frac{h}{2} \left[
    f(x_1) + 2f(x_2) + 2f(x_3) + 2f(x_4)+ \ldots + 2f(x_{n-1}) + f(x_n)
    \right] + \alpha h^p  + \beta h^q + \gamma h^r + \ldots \\
    &= T(h, n) + \alpha h^p  + \beta h^q + \gamma h^r + \ldots, 
    \tag{1}
\end{align*}


where $r > q > p > 0$ and for later use we introduced the notation

$$
    T(h, n) \equiv \frac{h}{2} \left[
    f(x_1) + 2f(x_2) + 2f(x_3) + 2f(x_4)+ \ldots + 2f(x_{n-1}) + f(x_n)
    \right] , 
    \tag{2}
$$

and

$$
    h = \frac{b-a}{n - 1}, \quad
    x_i = a + (i - 1) \, h, \quad i = 1, \ldots, n.
$$

Here $\alpha h^p$ is the *leading term* of *discretization error* of the
algorithm. The coefficients $\alpha$, $\beta$, $\gamma$, $\ldots$ are usually unknown. 

In [ ]:

using PyPlot

In [ ]:

"""
    ans = mytrapezoids(fun, a, b, n)

Numerically evaluate the integral int_a^b fun(x) dx using the
trapezoidal rule: I = h/2*(f_1 + 2f_2 + ... + 2f_{n-1} + f_n),
where h = (b - a)/(n - 1), x_i = a + (i - 1)*h, f_i = fun(x_i).
"""
function mytrapezoids(fun, a, b, n)
    h = (b - a)/(n - 1)
    s1 = fun(a) + fun(b)
    s2 = 0.0
    for i = 2:(n-1)
        s2 += fun(a + (i-1)*h)
    end
    return (s1 + 2*s2)*h/2
end


### Testing


#### Simple test #1: $\quad \displaystyle \int_1^2 \frac{\mathrm{d}x}{x} = \log(2)$

In [ ]:

fun1(x) = 1/x
a1 = 1.0
b1 = 2.0
np1 = 20
exact1 = log(2)

In [ ]:

round(abs(mytrapezoids(fun1, a1, b1, np1) - exact1), sigdigits=1)


#### Simple test #2: $\quad \displaystyle \int_0^1  e^x \, \mathrm{d}x = e - 1$

In [ ]:

fun2(x) = exp(x)
a2 = 0.0
b2 = 1.0
np2 = 100
exact2 = exp(1) - 1.0

In [ ]:

round(abs(mytrapezoids(fun2, a2, b2, np2) - exact2), sigdigits=1)


### Error of trapezoidal formula


A numerical experiment to determine the error of trapezoidal rule as a function of integration step $h$:

In [ ]:

fun3(x) = sin(x)
a3 = 0.0
b3 = pi
exact3 = 2.0

In [ ]:

ndp = 10           # number of tests
errt = zeros(ndp)  # absolute errors
hh = zeros(ndp);   # integration steps

In [ ]:

for k = 1:ndp 
    np = 2^k + 1
    hh[k] = (b3 - a3)/(np - 1)
    errt[k] = abs(mytrapezoids(fun3, a3, b3, np) - exact3)
end

In [ ]:

loglog(hh, errt, label="numerical experiment", marker=".")
p = 2
loglog(hh, hh .^ p, linestyle="dashed", label=L"hh^{%$p}")
grid(true)
xlabel("integration step")
ylabel("absolute error")
title("Error of trapezoidal rule")
legend();


## Richardson extrapolation


Recall that 
$$\int_a^b f(x) \, \mathrm{d}x = T(h, n) + \alpha h^2  + \beta h^q + \ldots, \tag{3}$$ 
where $T(h, n)$ is given by Eq. (2).

Next, we write down the trapezoidal rule with step $2h$. It will involve
only the nodes at $x_1$, $x_3$, $x_5$, $\ldots$, $x_{n-2}$, $x_n$:
$$\int_a^b f(x) \, \mathrm{d}x = h [ f(x_1) + 2f(x_3) + 2f(x_5) + \ldots + 2f(x_{n-2}) + f(x_n) ] + \alpha (2h)^2  + \beta (2h)^q + \ldots 
= T(2h) + 4\alpha h^2  + \beta'h^q + \ldots . \tag{4}$$

The leading error in Eq. (4) is 4 times the error in Eq. (3), since we doubled the step size. So we multiply both sides of Eq. (3) by 4,
$$
4\int_a^b f(x) \, \mathrm{d}x = 4T(h) + 4\alpha h^2 + \beta''h^q + \ldots,  \tag{5}
$$
and subtract Eq. (4) from Eq. (5), intending to
cancel out the leading error term. The result that we get is the
*Simpson's formula*:
$$
\int_a^b f(x) \, \mathrm{d}x  = \frac{1}{3}\left(4T(h) - T(2h) \right) + \beta'''h^q + \ldots .
$$

### Simpson's formula

In [ ]:

"""
    ans = mytrapezoids2(f, h)

Numerically evaluate the integral int_a^b fun(x) dx using the
trapezoidal rule: I = h/2*(f[1] + 2f[2] + ... + 2f[n-1] + f[n]),
where h = (b - a)/(n - 1), x_i = a + (i - 1)*h, f[i] = fun(x_i).
"""
function mytrapezoids2(fvec, h)
    s1 = fvec[1] + fvec[end]
    s2 = sum(fvec)
    return (2*s2 - s1)*h/2
end

In [ ]:

"""
    ans = mysimpsons2(fun, a, b, n)

Numerically evaluate the integral int_a^b fun(x) dx using the
Simpson's rule, where h = (b - a)/(n - 1), x_i = a + (i - 1)*h, fvec[i] = fun(x_i).
"""
function mysimpsons2(fun, a, b, n)
    x = range(a, b, n)
    fvec = fun.(x)
    h = x[2] - x[1]
    t1 = mytrapezoids2(fvec, h)
    t2 = mytrapezoids2(fvec[1:2:n], 2*h)
    return (4*t1 - t2)/3
end


#### Simple test #1: $\quad \displaystyle \int_1^2 \frac{\mathrm{d}x}{x} = \log(2)$

In [ ]:

np1 = 21
round(abs(mysimpsons2(fun1, a1, b1, np1) - exact1), sigdigits=1)


Approximately, how many nodes is necessary for trapezoidal rule to achieve the same precision?

In [ ]:

np1t = 501
round(abs(mytrapezoids(fun1, a1, b1, np1t) - exact1), sigdigits=1)


#### Simple test #2: $\quad \displaystyle \int_0^1  e^x \, \mathrm{d}x = e - 1$

In [ ]:

np2 = 101
round(abs(mysimpsons2(fun2, a2, b2, np2) - exact2), sigdigits=1)

In [ ]:

np2t = 34001
round(abs(mytrapezoids(fun2, a2, b2, np2t) - exact2), sigdigits=1)